# KNN Recommender 

👉 K-Nearest-Neighbors (KNN) models can be used to model and make predictions, but they can alternatively be utilized to find the closest points in a dataset.  

👨🏻‍🏫 In this recap, we will use a KNN model to create a basic music recommender system.

In [ ]:
# Regression Metrics
# R2: goodness of fit
# Mean square error
# Mean absolute error
# Max error
# RMSE

In [1]:
import pandas as pd

url = 'https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_spotify_data.csv'

# Using pandas, load the data from the provided URL
df = pd.read_csv(url)
df.head()

,name,artists,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness,tempo
0,We're For The Dark - Remastered 2010,['Badfinger'],22,0.678,0.559,0.432,0,3,0.0727,-12.696,0.0334,117.674
1,Sixty Years On - Piano Demo,['Elton John'],25,0.456,0.259,0.368,0,6,0.1560,-10.692,0.0280,143.783
2,Got to Find Another Way,['The Guess Who'],21,0.433,0.833,0.724,0,0,0.1700,-9.803,0.0378,84.341
3,Feelin' Alright - Live At The Fillmore East/1970,['Joe Cocker'],22,0.436,0.870,0.914,0,5,0.8550,-6.955,0.0610,174.005
4,Caravan - Take 7,['Van Morrison'],23,0.669,0.564,0.412,0,7,0.4010,-13.095,0.0679,78.716


🎯 Let's find songs that are "similar" to Queen's mythical *Another One Bites the Dust*.

In [2]:
queen_song = df.iloc[4295:4296] # Another One Bites the Dust - Queen

queen_song

,name,artists,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness,tempo
4295,Another One Bites The Dust - Live at Wembley '86,['Queen'],29,0.534,0.114,0.984,0,4,0.982,-5.058,0.297,115.991


## 1. Calculating the distances

👇 First, fit a KNN on the (scaled) dataset. 

Since we are only concerned with the similarity of features between the songs, it doesn't matter which target the model is fitted on.

In [7]:
X_num = df.select_dtypes(exclude=["object"])
y = X_num["tempo"]
X = X_num.drop(columns=["tempo"])
X.head()

,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness
0,22,0.678,0.559,0.432,0,3,0.0727,-12.696,0.0334
1,25,0.456,0.259,0.368,0,6,0.1560,-10.692,0.0280
2,21,0.433,0.833,0.724,0,0,0.1700,-9.803,0.0378
3,22,0.436,0.870,0.914,0,5,0.8550,-6.955,0.0610
4,23,0.669,0.564,0.412,0,7,0.4010,-13.095,0.0679


In [8]:
y

0       117.674
1       143.783
2        84.341
3       174.005
4        78.716
         ...   
9995    105.029
9996    137.936
9997     91.688
9998     75.055
9999     94.991
Name: tempo, Length: 10000, dtype: float64

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
X_scaled.head()

,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness
0,0.255814,0.687627,0.561245,0.432,0.0,0.272727,0.0727,0.774548,0.034901
1,0.290698,0.462475,0.260040,0.368,0.0,0.545455,0.1560,0.807362,0.029258
2,0.244186,0.439148,0.836345,0.724,0.0,0.000000,0.1700,0.821918,0.039498
3,0.255814,0.442191,0.873494,0.914,0.0,0.454545,0.8550,0.868551,0.063741
4,0.267442,0.678499,0.566265,0.412,0.0,0.636364,0.4010,0.768015,0.070951


In [15]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
knn.fit(X_scaled, y)

KNeighborsRegressor()

Check out the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor.kneighbors)

## 2. Passing the new point

👇 You can now pass a new point to the KNN model and find its closest point.

In [16]:
queen_song

,name,artists,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness,tempo
4295,Another One Bites The Dust - Live at Wembley '86,['Queen'],29,0.534,0.114,0.984,0,4,0.982,-5.058,0.297,115.991


In [18]:
queen_song_new = queen_song.drop(columns=["artists", "name", "tempo"])
queen_song_new

,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness
4295,29,0.534,0.114,0.984,0,4,0.982,-5.058,0.297


In [21]:
queen_song_new_df = pd.DataFrame(scaler.transform(queen_song_new), columns=queen_song_new.columns)
queen_song_new_df

,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness
0,0.337209,0.541582,0.114458,0.984,0.0,0.363636,0.982,0.899612,0.310345


In [40]:
knn.kneighbors(queen_song_new_df, n_neighbors=2)

(array([[0.        , 0.34255512]]), array([[4295, 1164]]))

In [41]:
df.iloc[4295]

name            Another One Bites The Dust - Live at Wembley '86
artists                                                ['Queen']
popularity                                                    29
danceability                                               0.534
valence                                                    0.114
energy                                                     0.984
explicit                                                       0
key                                                            4
liveness                                                   0.982
loudness                                                  -5.058
speechiness                                                0.297
tempo                                                    115.991
Name: 4295, dtype: object

In [42]:
df.iloc[1164]

name            Hi, Hi, Hi - Live / Remastered
artists                              ['Wings']
popularity                                  27
danceability                             0.219
valence                                  0.162
energy                                   0.939
explicit                                     0
key                                          4
liveness                                 0.993
loudness                                -9.275
speechiness                              0.226
tempo                                  140.832
Name: 1164, dtype: object

## 3. Making a playlist!

👇 Make a playlist with 10 songs based on Queen's *Another One Bites the Dust*, sorted by increasing tempo.

In [43]:
queen_song

,name,artists,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness,tempo
4295,Another One Bites The Dust - Live at Wembley '86,['Queen'],29,0.534,0.114,0.984,0,4,0.982,-5.058,0.297,115.991


In [53]:
list(knn.kneighbors(queen_song_new_df, n_neighbors=11)[1][0])

[4295, 1164, 8607, 1761, 704, 1614, 2233, 1211, 3307, 1221, 2289]

In [58]:
df.iloc[list(knn.kneighbors(queen_song_new_df, n_neighbors=11)[1][0])].sort_values(by="tempo")

,name,artists,popularity,danceability,valence,energy,explicit,key,liveness,loudness,speechiness,tempo
3307,Graveyard,['Butthole Surfers'],27,0.504,0.1350,0.949,0,7,0.913,-8.797,0.0385,98.128
704,"It Ain't Me, Babe - Live at LA Forum, Inglewoo...",['Bob Dylan'],23,0.455,0.3080,0.981,0,7,0.995,-6.409,0.1830,100.490
1761,Liar,['The Damned'],25,0.348,0.2030,0.939,0,4,0.838,-11.540,0.0745,107.064
1211,A Light In The Black,['Rainbow'],32,0.334,0.0936,0.982,0,4,0.753,-10.190,0.0735,109.414
4295,Another One Bites The Dust - Live at Wembley '86,['Queen'],29,0.534,0.1140,0.984,0,4,0.982,-5.058,0.2970,115.991
8607,Cheat Codes,['Nitro Fun'],51,0.626,0.1460,0.960,0,4,0.894,-4.234,0.0837,128.001
1164,"Hi, Hi, Hi - Live / Remastered",['Wings'],27,0.219,0.1620,0.939,0,4,0.993,-9.275,0.2260,140.832
2289,You've Got Another Thing Coming - Live from th...,['Judas Priest'],27,0.382,0.3270,0.929,0,6,0.966,-5.921,0.0568,144.354
2233,YYZ - Live In Canada / 1980,['Rush'],26,0.334,0.2780,0.911,0,4,0.937,-12.017,0.0642,145.905
1221,Southside Shuffle - Live,['The J. Geils Band'],20,0.420,0.3540,0.923,0,7,0.960,-8.690,0.2800,153.333
